In [ ]:
# code for paper The Emergence of Benford's Distribution in Directed Networks: A Study of a Multiplicative Evolution Mechanism

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import Counter
from scipy.stats import chisquare

# ==================== Parameters ====================
# Network basic parameters
N_NODES = 10000
INITIAL_DEGREE_MIN = 1
INITIAL_DEGREE_MAX = 3

# Iteration parameters
N_ITERATIONS = 8                        # change iterations here
MULTIPLY_FACTOR_MIN = 0.5
MULTIPLY_FACTOR_MAX = 4

# Random seed
RANDOM_SEED = 42

# Visualization parameters
HIST_BINS = 50
SHOW_NETWORK_SAMPLE = 100

# Benford test parameters
ALPHA = 0.05
# ===================================================

def create_initial_graph(n_nodes=N_NODES):
    """Create an empty directed graph with n_nodes nodes."""
    G = nx.DiGraph()
    G.add_nodes_from(range(n_nodes))
    return G

def add_initial_edges(G, min_degree=INITIAL_DEGREE_MIN, max_degree=INITIAL_DEGREE_MAX):
    """Add initial random in-degree edges for every node."""
    nodes = list(G.nodes())
    print(f"Adding initial edges (in-degree range: {min_degree}-{max_degree})...")
    for target in nodes:
        k = np.random.randint(min_degree, max_degree + 1)
        candidates = [n for n in nodes if n != target]
        sources = np.random.choice(candidates, size=k, replace=False)
        for s in sources:
            G.add_edge(s, target)
    return G

def multiply_degrees_iteration(G, factor_min=MULTIPLY_FACTOR_MIN,
                               factor_max=MULTIPLY_FACTOR_MAX):
    """Perform one multiplicative update iteration."""
    for node in G.nodes():
        current = G.in_degree(node)
        if current > 0:
            factor = np.random.uniform(factor_min, factor_max)
            new_deg = max(1, int(current * factor))
        else:
            new_deg = 1
        adjust_node_degree(G, node, new_deg)
    return G

def adjust_node_degree(G, target, target_degree):
    """Adjust the in-degree of a single node to the target value."""
    current = G.in_degree(target)
    others = [n for n in G.nodes() if n != target]

    if target_degree > current:
        needed = target_degree - current
        existing = set(G.predecessors(target))
        available = [n for n in others if n not in existing]
        for _ in range(needed):
            if available:
                s = np.random.choice(available)
                available.remove(s)
            else:
                s = np.random.choice(others)
            G.add_edge(s, target)

    elif target_degree < current:
        to_del = current - target_degree
        preds = list(G.predecessors(target))
        for s in np.random.choice(preds, size=to_del, replace=False):
            G.remove_edge(s, target)

def simulate_network_growth(n_nodes=N_NODES,
                            min_degree=INITIAL_DEGREE_MIN,
                            max_degree=INITIAL_DEGREE_MAX,
                            iterations=N_ITERATIONS,
                            factor_min=MULTIPLY_FACTOR_MIN,
                            factor_max=MULTIPLY_FACTOR_MAX):
    """Run the full simulation and return the graph after every iteration."""
    print("=== Network simulation start ===")
    # 0th iteration = initial graph
    G = create_initial_graph(n_nodes)
    G = add_initial_edges(G, min_degree, max_degree)
    graphs = [G]

    # Iterations 1..iterations
    for i in range(iterations):
        print(f"Iteration {i+1}...")
        G = multiply_degrees_iteration(G.copy(), factor_min, factor_max)
        graphs.append(G)
    print('iteration done!')
    return graphs          # len == iterations + 1

def first_digit_distribution(values):
    """Return observed Benford frequency list for 1..9."""
    first_digits = [int(str(abs(v))[0]) for v in values if v > 0]
    if not first_digits:
        return [0]*9
    counter = Counter(first_digits)
    total = len(first_digits)
    return [counter.get(d, 0)/total for d in range(1, 10)]

def plot_all_iterations(graphs):
    """Plot 3×3 grid of Benford comparisons for iterations 0..8."""
    benford_expected = [np.log10(1+1/d) for d in range(1, 10)]
    digits = np.arange(1, 10)

    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    axes = axes.flatten()

    for idx, G in enumerate(graphs):
        degrees = [G.in_degree(n) for n in G.nodes()]
        observed = first_digit_distribution(degrees)

        ax = axes[idx]
        ax.bar(digits, observed, alpha=0.7, label='Observed')
        ax.plot(digits, benford_expected, 'ro-', label='Benford')
        ax.set_title(f'Iteration {idx}')
        ax.set_xticks(digits)
        ax.set_ylim(0, 0.45)
        ax.grid(alpha=0.3)
        if idx == 0:
            ax.legend()

    plt.tight_layout()
    plt.show()

def main():
    print("=== Benford-law Network Simulation ===")
    if RANDOM_SEED is not None:
        np.random.seed(RANDOM_SEED)
        random.seed(RANDOM_SEED)

    graphs = simulate_network_growth()
    plot_all_iterations(graphs)
    test_last_three(graphs)

if __name__ == "__main__":
    main()